In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access the variables
subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP")
workspace_name = os.getenv("WORKSPACE_NAME")

# Connect to ML workspace using InteractiveBrowserCredential
credential = DefaultAzureCredential()

# Create an MLClient object
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

In [3]:
# Get and download the model
model = ml_client.models.get(name="german-credit-card-hsg", version="1")

In [4]:
from azure.ai.ml.entities import Environment

# Load environment from a conda YAML file
myenv = Environment(
    name="myenv", 
    conda_file="environment.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04")

In [5]:
from azure.ai.ml.entities import ManagedOnlineDeployment, ManagedOnlineEndpoint
from azure.ai.ml.constants import AssetTypes

# Define a unique endpoint name
endpoint_name = "german-credit-hsg-endpoint"

# Create endpoint configuration
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Predict credit risk with sklearn",
    auth_mode="key"
)

# Create the endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://german-credit-hsg-endpoint.switzerlandnorth.inference.ml.azure.com/score', 'openapi_uri': 'https://german-credit-hsg-endpoint.switzerlandnorth.inference.ml.azure.com/swagger.json', 'name': 'german-credit-hsg-endpoint', 'description': 'Predict credit risk with sklearn', 'tags': {}, 'properties': {'createdBy': 'Simone Finelli', 'createdAt': '2024-09-26T19:50:16.504635+0000', 'lastModifiedAt': '2024-09-26T19:50:16.504635+0000', 'azureml.onlineendpointid': '/subscriptions/b5b33273-782e-496a-964f-72acfc982719/resourcegroups/hsg-lesson-rg/providers/microsoft.machinelearningservices/workspaces/hsg-lesson-aml/onlineendpoints/german-credit-hsg-endpoint', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/b5b33273-782e-496a-964f-72acfc982719/providers/Microsoft.MachineLearningServices/locations/switzerlandnorth/mfeOperationsStatus/oeidp:d3bd7ea2-38ec-4bd6-a972-

In [6]:
%%writefile scripts/score.py

import json
import os
import numpy as np
import pandas as pd
import joblib
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.identity import DefaultAzureCredential

MODEL_NAME = 'german-credit-card-hsg'

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # Please provide your model's folder name if there is one
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "model.pkl")
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        input_df = pd.DataFrame.from_dict(data)
        proba = model.predict_proba(input_df)
        result = {"predict_proba": proba.tolist()}
        return result
    except Exception as e:
        return str(e)

Overwriting scripts/score.py


In [6]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment, CodeConfiguration

# Create an inference configuration
code_config = CodeConfiguration(
    code="./scripts",  # Folder where the scoring script is located
    scoring_script="score.py"
)

# Deploy the model
deployment = ManagedOnlineDeployment(
    name="credit-risk-deploy",
    endpoint_name=endpoint_name,
    model=model,  # The model you downloaded
    code_configuration=code_config,
    instance_type="Standard_DS3_v2",
    instance_count=1,
    environment=myenv
)

# Deploy the model
ml_client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint german-credit-hsg-endpoint exists


.......................................................................